# Import

In [1]:
# calcium traces exploration - simple commands in pandas
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.io
import scipy.stats
import os
import pickle
from matplotlib import cm
from sklearn.model_selection import KFold, cross_validate, StratifiedKFold, GridSearchCV, cross_val_score
from sklearn.linear_model import RidgeClassifierCV
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn import decomposition
from sklearn.preprocessing import scale
from sklearn.decomposition import PCA
import random
import copy

# permutation tests: first principal component (airpuff, sessions)

In [ ]:
"""
This script includes:
    -Permutation tests to compare different time windows around points of interest using first principal 
    component for airpuff & non-airpuff sessions (using sessions & trials as datapoints)
    -Permutation tests to compare airpuff vs non-airpuff sessions for each time window using first principal
    component (using sessions & trials as datapoints)

---------
Python 3.8.5

Dependencies:
    -numpy 1.19.2
    -pandas 1.1.3
    -scipy 1.6.3
    -matplotlib 3.3.2
    -seaborn 0.11.0
    -sklearn 0.0
    -csv, os, pickle, random, copy
"""

In [4]:
animal_folder_name = "/Users/felicia/Documents/CFIN/232/232_traces/"
session_list = ['04','06','07','08','10','11','12','13'] # 232
#session_list = ['06','07','08','09','10','11','12','13'] # 233

In [5]:
# initiate empty lists
first_var_start = []
first_var_stim_onset = []
first_var_resp = []
first_var_punish = []
first_var_reward = []

# for all airpuff sessions:
for session in range(14,20):

    # LOAD CSV CALCIUM TRACES
    folder_name = "Calcium_Traces/" 
    filename = "2021-05-{0}.csv".format(session)
    df = pd.read_csv(os.path.join(animal_folder_name,folder_name, filename))
    n_rows, n_columns = df.shape

    # create time array
    df_times = df[' '][1:-1]
    df_num_time = pd.to_numeric(df_times, downcast='float')
    time_array = df_num_time.to_numpy()
    tot_timepoints = len(time_array)

    # check time sampling - i.e. if every data point is taken at the same "distance" in time from the next - or previous
    time_sampling = np.zeros(shape=(len(time_array)-1,))

    #12420 timepoints sampled at 100msec. means that every 10 of those timepoints is 1 sec
    sampling_rate = 0.1
    tot_secs = int(tot_timepoints * sampling_rate)

    #create channels array
    n_channels = n_columns-1
    channels_array = np.zeros(shape=(tot_timepoints, n_channels)) # initiate empty array
    # fill empty channels array
    for i in range(n_channels):
        col_name = df.columns[i+1]
        df_channel = df[col_name][1:-1]
        df_num_channel = pd.to_numeric(df_channel, downcast='float')
        channels_array[:,i] = df_num_channel.to_numpy()

    # LOAD BEHAVIOURAL DATA
    #folder_name = "trial_info"
    #bhv_info_filename = "trial_info_2021-05-{0}.mat".format(session)
    #trial_info = scipy.io.loadmat(os.path.join(animal_folder_name,folder_name, bhv_info_filename))

    # extract information from trial_info: in matlab this is a struct with the fields:
    #trials_start_time = trial_info['trial_info'][0,0][0].ravel()
    #trials_stimulus_on = trial_info['trial_info'][0,0][1].ravel()
    #trials_response = trial_info['trial_info'][0,0][2].ravel()
    #trials_is_right_lick = trial_info['trial_info'][0,0][3].ravel()
    #trials_is_reward = trial_info['trial_info'][0,0][4].ravel()
    #trials_end_time = trial_info['trial_info'][0,0][5].ravel()
    #n_trials = len(trials_end_time) # number of trials
    
    # load behavioural data (.pkl file)
    input_filename = "trial_info/trial_info_2021-05-{0}.pkl".format(session)
    with open(os.path.join(animal_folder_name,input_filename), 'rb') as fp:
         trial_info = pickle.load(fp)

    # extract information for trial_info: in PYTHON
    trials_start_time = np.array(trial_info['start_time'])
    trials_stimulus_on = np.array(trial_info['stimulus_on'])
    trials_response = np.array(trial_info['response'])
    trials_is_right_lick = np.array(trial_info['is_right_lick'])
    trials_is_reward = np.array(trial_info['is_reward'])
    trials_end_time = np.array(trial_info['end_trial'])
    n_trials = len(trials_end_time)

    # 500msec between start and stimulus onset
    new_start = [] # initiate empty list
    new_end = [] # initiate empty list
    new_start.append(trials_stimulus_on[0]-0.5) # append first start
    new_end.append(trials_stimulus_on[1]-0.5) # append first end
    for i in range(1,n_trials-1): # append start & end times for each trial (except last)
        new_start.append(trials_stimulus_on[i]-0.5)
        new_end.append(trials_stimulus_on[i+1]-0.5)
    new_start.append(trials_stimulus_on[n_trials-1]-0.5) # append last start
    new_end.append(trials_end_time[n_trials-1]) # append last end
    new_start = np.array(new_start) # turn list to array
    new_end = np.array(new_end)
    n_trials = len(new_end)
    
    # where was the stimulus? we don't have this info. But it can be reconstructed 
    stimulus_reconstruction = np.zeros(shape=(n_trials,))
    right_choice = np.where(trials_is_reward==1)[0]
    stimulus_reconstruction[right_choice] = trials_is_right_lick[right_choice]
    wrong_choice = np.where(trials_is_reward==0)[0]
    stimulus_reconstruction[wrong_choice] = 1 - trials_is_right_lick[wrong_choice]

    # split traces data into trials
    X = {}
    for i in range(n_trials-1):
        lower_bound = np.min(np.where(time_array>=new_start[i]))
        upper_bound = np.max(np.where(time_array<=new_end[i]))
        X[i] = channels_array[lower_bound:upper_bound,:]
        
        
    ###### across all trials for this session
    n_components = 7
    start_timepoint = 0
    n_after_start = 7
    n_before_stim = 2
    n_after_stim = 5
    n_before_resp = 5
    n_after_resp = 2
    n_before_pun = 2
    n_after_pun = 5
    n_before_rew = 2
    n_after_rew = 5
    time_window = n_before_stim+n_after_stim
    airpuff = 2

    # start of trial
    variances_start = []
    for idx in X:
        x = X[idx][start_timepoint:n_after_start]
        if len(x)>6: # make sure that time window is not too small
            covar_matrix = PCA(n_components=n_components)
            covar_matrix.fit(x)
            variance = covar_matrix.explained_variance_ratio_ #calculate variance ratios
            var_trial = np.cumsum(np.round(covar_matrix.explained_variance_ratio_, decimals=3))
        variances_start.append(var_trial)
    var_start = np.mean(variances_start, axis=0)

    # stimulus onset
    variances_stim_onset = []
    for idx in X:
        stim_timepoint = int((trials_stimulus_on[idx] - new_start[idx]) /sampling_rate)
        x = X[idx][stim_timepoint-n_before_stim:stim_timepoint + n_after_stim]
        if len(x)>6: # make sure that time window is not too small
            covar_matrix = PCA(n_components=n_components)
            covar_matrix.fit(x)
            variance = covar_matrix.explained_variance_ratio_ #calculate variance ratios
            var_trial = np.cumsum(np.round(covar_matrix.explained_variance_ratio_, decimals=3))
        variances_stim_onset.append(var_trial)
    var_stim_onset = np.mean(variances_stim_onset, axis=0)

    # response
    variances_resp = []
    for idx in X:
        resp_timepoint = int((trials_response[idx] - new_start[idx]) / sampling_rate)
        x = X[idx][resp_timepoint - n_before_resp:resp_timepoint+n_after_resp]
        if len(x)>6: # make sure that time window is not too small
            covar_matrix = PCA(n_components=n_components)
            covar_matrix.fit(x)
            variance = covar_matrix.explained_variance_ratio_ #calculate variance ratios
            var_trial = np.cumsum(np.round(covar_matrix.explained_variance_ratio_, decimals=3))
        variances_resp.append(var_trial)
    var_resp = np.mean(variances_resp, axis=0)

    # punishment
    variances_punish = []
    for idx in X:
        # only punishment trials
        if trials_is_reward[idx]==0:
            # timepoint for response
            resp_airpuff_timepoint = int((trials_response[idx] - new_start[idx]) / sampling_rate) + airpuff
            x = X[idx][resp_timepoint-n_before_pun:resp_timepoint+n_after_pun]
            if len(x)>6: # make sure that time window is not too small
                covar_matrix = PCA(n_components=n_components)
                covar_matrix.fit(x)
                variance = covar_matrix.explained_variance_ratio_ #calculate variance ratios
                var_trial = np.cumsum(np.round(covar_matrix.explained_variance_ratio_, decimals=3))
        variances_punish.append(var_trial)
    var_punish = np.mean(variances_punish, axis=0)
    
    # reward
    variances_reward = []
    for idx in X:
        # only reward trials
        if trials_is_reward[idx]==1:
            # timepoint for response
            resp_timepoint = int((trials_end_time[idx] - new_start[idx]) / sampling_rate)
            x = X[idx][resp_timepoint-n_before_rew:resp_timepoint+n_after_rew]
            if len(x)>6: # make sure that time window is not too small
                covar_matrix = PCA(n_components=n_components)
                covar_matrix.fit(x)
                variance = covar_matrix.explained_variance_ratio_ #calculate variance ratios
                var_trial = np.cumsum(np.round(covar_matrix.explained_variance_ratio_, decimals=3))
        variances_reward.append(var_trial)
    var_reward = np.mean(variances_reward, axis=0)

    first_var_start.append(var_start[0])
    first_var_stim_onset.append(var_stim_onset[0])
    first_var_resp.append(var_resp[0])
    first_var_punish.append(var_punish[0])
    first_var_reward.append(var_reward[0])

In [6]:
# change variable1 and variable2 to compute permutation and t-tests between them:
# first_var_start  first_var_stim_onset  first_var_resp  first_var_punish  first_var_reward
variable1 = first_var_punish
variable2 = first_var_reward

# Compute the ground truth absolute difference Δτ between τ₁ and τ₂ from your two variables:
gT = np.abs(np.average(variable1) - np.average(variable2))

# Pool your variables into one single distribution:
pV = list(variable1) + list(variable2)

# Randomly sample (also called bootstrapping) without replacement two distributions 
# with the size equal to the original distributions from this pooled distribution 
# to compute the absolute difference RΔτ of your metric between your two permuted samples:
# and repeat this p times:

# Copy pooled distribution:
pS = copy.copy(pV)
# Initialize permutation:
pD = []
# Define p (number of permutations):
p=1000
# Permutation loop:
for i in range(0,p):
  # Shuffle the data:
    random.shuffle(pS)
    # Compute permuted absolute difference of your two sampled distributions and store it in pD:
    pD.append(np.abs(np.average(pS[0:int(len(pS)/2)]) - np.average(pS[int(len(pS)/2):])))
    
# Finally, the proportion of permuted differences higher than your ground truth difference 
# is your significance value:
p_val = len(np.where(pD>=gT)[0])/p
print("permutation test: pvalue =", p_val)
# Compare with parametric t-test:
print("parametric t-test:", scipy.stats.stats.ttest_ind(variable1, variable2))

permutation test: pvalue = 0.985
parametric t-test: Ttest_indResult(statistic=-0.31067269546489856, pvalue=0.7624282505406907)


# permutation tests: first principal component (airpuff, trials)

In [7]:
# initiate empty lists
variances_start = []
variances_stim_onset = []
variances_resp = []
variances_punish = []
variances_reward = []
tot_trials = 0

# for all airpuff sessions:
for session in range(14,20):

    # LOAD CSV CALCIUM TRACES
    folder_name = "Calcium_Traces/" 
    filename = "2021-05-{0}.csv".format(session)
    df = pd.read_csv(os.path.join(animal_folder_name,folder_name, filename))
    n_rows, n_columns = df.shape

    # create time array
    df_times = df[' '][1:-1]
    df_num_time = pd.to_numeric(df_times, downcast='float')
    time_array = df_num_time.to_numpy()
    tot_timepoints = len(time_array)

    # check time sampling - i.e. if every data point is taken at the same "distance" in time from the next - or previous
    time_sampling = np.zeros(shape=(len(time_array)-1,))

    #12420 timepoints sampled at 100msec. means that every 10 of those timepoints is 1 sec
    sampling_rate = 0.1
    tot_secs = int(tot_timepoints * sampling_rate)

    #create channels array
    n_channels = n_columns-1
    channels_array = np.zeros(shape=(tot_timepoints, n_channels)) # initiate empty array
    # fill empty channels array
    for i in range(n_channels):
        col_name = df.columns[i+1]
        df_channel = df[col_name][1:-1]
        df_num_channel = pd.to_numeric(df_channel, downcast='float')
        channels_array[:,i] = df_num_channel.to_numpy()

    # LOAD BEHAVIOURAL DATA
    #folder_name = "trial_info"
    #bhv_info_filename = "trial_info_2021-05-{0}.mat".format(session)
    #trial_info = scipy.io.loadmat(os.path.join(animal_folder_name,folder_name, bhv_info_filename))

    # extract information from trial_info: in matlab this is a struct with the fields:
    #trials_start_time = trial_info['trial_info'][0,0][0].ravel()
    #trials_stimulus_on = trial_info['trial_info'][0,0][1].ravel()
    #trials_response = trial_info['trial_info'][0,0][2].ravel()
    #trials_is_right_lick = trial_info['trial_info'][0,0][3].ravel()
    #trials_is_reward = trial_info['trial_info'][0,0][4].ravel()
    #trials_end_time = trial_info['trial_info'][0,0][5].ravel()
    #n_trials = len(trials_end_time) # number of trials
    #tot_trials = tot_trials + n_trials # total number of trials across all sessions
    
    # load behavioural data (.pkl file)
    input_filename = "trial_info/trial_info_2021-05-{0}.pkl".format(session)
    with open(os.path.join(animal_folder_name,input_filename), 'rb') as fp:
         trial_info = pickle.load(fp)

    # extract information for trial_info: in PYTHON
    trials_start_time = np.array(trial_info['start_time'])
    trials_stimulus_on = np.array(trial_info['stimulus_on'])
    trials_response = np.array(trial_info['response'])
    trials_is_right_lick = np.array(trial_info['is_right_lick'])
    trials_is_reward = np.array(trial_info['is_reward'])
    trials_end_time = np.array(trial_info['end_trial'])
    n_trials = len(trials_end_time)
    tot_trials = tot_trials + n_trials # total number of trials across all sessions

    # 500msec between start and stimulus onset
    new_start = [] # initiate empty list
    new_end = [] # initiate empty list
    new_start.append(trials_stimulus_on[0]-0.5) # append first start
    new_end.append(trials_stimulus_on[1]-0.5) # append first end
    for i in range(1,n_trials-1): # append start & end times for each trial (except last)
        new_start.append(trials_stimulus_on[i]-0.5)
        new_end.append(trials_stimulus_on[i+1]-0.5)
    new_start.append(trials_stimulus_on[n_trials-1]-0.5) # append last start
    new_end.append(trials_end_time[n_trials-1]) # append last end
    new_start = np.array(new_start) # turn list to array
    new_end = np.array(new_end)
    n_trials = len(new_end)
    
    # where was the stimulus? we don't have this info. But it can be reconstructed 
    stimulus_reconstruction = np.zeros(shape=(n_trials,))
    right_choice = np.where(trials_is_reward==1)[0]
    stimulus_reconstruction[right_choice] = trials_is_right_lick[right_choice]
    wrong_choice = np.where(trials_is_reward==0)[0]
    stimulus_reconstruction[wrong_choice] = 1 - trials_is_right_lick[wrong_choice]

    X = {}

    for i in range(n_trials-1):
        lower_bound = np.min(np.where(time_array>=new_start[i]))
        upper_bound = np.max(np.where(time_array<=new_end[i]))
        X[i] = channels_array[lower_bound:upper_bound,:]
        
        
    ###### across all trials for this session
    n_components = 7
    start_timepoint = 0
    n_after_start = 7
    n_before_stim = 2
    n_after_stim = 5
    n_before_resp = 5
    n_after_resp = 2
    n_before_pun = 2
    n_after_pun = 5
    n_before_rew = 2
    n_after_rew = 5
    time_window = n_before_stim+n_after_stim
    airpuff = 2

    # start of trial
    for idx in X:
        x = X[idx][start_timepoint:n_after_start]
        if len(x)>6: # make sure that time window is not too small
            covar_matrix = PCA(n_components=n_components)
            covar_matrix.fit(x)
            variance = covar_matrix.explained_variance_ratio_ #calculate variance ratios
            var_trial = np.cumsum(np.round(covar_matrix.explained_variance_ratio_, decimals=3))
        variances_start.append(var_trial[0])

    # stimulus onset
    for idx in X:
        stim_timepoint = int((trials_stimulus_on[idx] - new_start[idx]) /sampling_rate)
        x = X[idx][stim_timepoint-n_before_stim:stim_timepoint + n_after_stim]
        if len(x)>6: # make sure that time window is not too small
            covar_matrix = PCA(n_components=n_components)
            covar_matrix.fit(x)
            variance = covar_matrix.explained_variance_ratio_ #calculate variance ratios
            var_trial = np.cumsum(np.round(covar_matrix.explained_variance_ratio_, decimals=3))
        variances_stim_onset.append(var_trial[0])

    # response
    for idx in X:
        resp_timepoint = int((trials_response[idx] - new_start[idx]) / sampling_rate)
        x = X[idx][resp_timepoint - n_before_resp:resp_timepoint+n_after_resp]
        if len(x)>6: # make sure that time window is not too small
            covar_matrix = PCA(n_components=n_components)
            covar_matrix.fit(x)
            variance = covar_matrix.explained_variance_ratio_ #calculate variance ratios
            var_trial = np.cumsum(np.round(covar_matrix.explained_variance_ratio_, decimals=3))
        variances_resp.append(var_trial[0])

    # punishment
    for idx in X:
        # only punishment trials
        if trials_is_reward[idx]==0:
            # timepoint for response
            resp_airpuff_timepoint = int((trials_response[idx] - new_start[idx]) / sampling_rate) + airpuff
            x = X[idx][resp_timepoint-n_before_pun:resp_timepoint+n_after_pun]
            if len(x)>6: # make sure that time window is not too small
                covar_matrix = PCA(n_components=n_components)
                covar_matrix.fit(x)
                variance = covar_matrix.explained_variance_ratio_ #calculate variance ratios
                var_trial = np.cumsum(np.round(covar_matrix.explained_variance_ratio_, decimals=3))
        variances_punish.append(var_trial[0])
    
    # reward
    for idx in X:
        # only reward trials
        if trials_is_reward[idx]==1:
            # timepoint for response
            resp_timepoint = int((trials_end_time[idx] - new_start[idx]) / sampling_rate)
            x = X[idx][resp_timepoint-n_before_rew:resp_timepoint+n_after_rew]
            if len(x)>6: # make sure that time window is not too small
                covar_matrix = PCA(n_components=n_components)
                covar_matrix.fit(x)
                variance = covar_matrix.explained_variance_ratio_ #calculate variance ratios
                var_trial = np.cumsum(np.round(covar_matrix.explained_variance_ratio_, decimals=3))
        variances_reward.append(var_trial[0])

In [14]:
# change variable1 and variable2 to compute permutation and t-tests between them:
# variances_start  variances_stim_onset  variances_resp  variances_punish  variances_reward
variable1 = variances_punish
variable2 = variances_reward

# Compute the ground truth absolute difference Δτ between τ₁ and τ₂ from your two variables:
gT = np.abs(np.average(variable1) - np.average(variable2))

# Pool your variables into one single distribution:
pV = list(variable1) + list(variable2)

# Randomly sample (also called bootstrapping) without replacement two distributions 
# with the size equal to the original distributions from this pooled distribution 
# to compute the absolute difference RΔτ of your metric between your two permuted samples:
# and repeat this p times:

# Copy pooled distribution:
pS = copy.copy(pV)
# Initialize permutation:
pD = []
# Define p (number of permutations):
p=1000
# Permutation loop:
for i in range(0,p):
  # Shuffle the data:
    random.shuffle(pS)
    # Compute permuted absolute difference of your two sampled distributions and store it in pD:
    pD.append(np.abs(np.average(pS[0:int(len(pS)/2)]) - np.average(pS[int(len(pS)/2):])))
    
    
# multiple testing correction?
    
# Finally, the proportion of permuted differences higher than your ground truth difference 
# is your significance value:
p_val = len(np.where(pD>=gT)[0])/p
print("permutation test: pvalue =", p_val)
# Compare with parametric t-test:
print("parametric t-test:", scipy.stats.stats.ttest_ind(variable1, variable2))

permutation test: pvalue = 0.009
parametric t-test: Ttest_indResult(statistic=array([2.13176585, 4.03497078, 2.51066894, 0.91667998, 2.54266201,
       1.40233269, 1.40233269]), pvalue=array([3.42712779e-02, 7.82353350e-05, 1.28586449e-02, 3.60436884e-01,
       1.17737222e-02, 1.62397768e-01, 1.62397768e-01]))


# permutation tests: first principal component (no airpuff, sessions)

In [15]:
# initiate empty lists
first_var_start = []
first_var_stim_onset = []
first_var_resp = []
first_var_punish = []
first_var_reward = []

# for all non-airpuff sessions:
for session in session_list:

    # load csv calcium traces
    folder_name = "Calcium_Traces/" 
    filename = "2021-05-{0}.csv".format(session)
    df = pd.read_csv(os.path.join(animal_folder_name,folder_name, filename))
    n_rows, n_columns = df.shape

    # create time array
    df_times = df[' '][1:-1]
    df_num_time = pd.to_numeric(df_times, downcast='float')
    time_array = df_num_time.to_numpy()
    tot_timepoints = len(time_array)

    # check time sampling - i.e. if every data point is taken at the same "distance" in time from the next - or previous
    time_sampling = np.zeros(shape=(len(time_array)-1,))
    for i in range(1,len(time_array)):
        time_sampling[i-1] = time_array[i]-time_array[i-1]

    #12420 timepoints sampled at 100msec. means that every 10 of those timepoints is 1 sec
    sampling_rate = 0.1
    tot_secs = int(tot_timepoints * sampling_rate)

    #create channels array
    n_channels = n_columns-1
    channels_array = np.zeros(shape=(tot_timepoints, n_channels)) # initiate empty array
    # fill empty channels array
    for i in range(n_channels):
        col_name = df.columns[i+1]
        df_channel = df[col_name][1:-1]
        df_num_channel = pd.to_numeric(df_channel, downcast='float')
        channels_array[:,i] = df_num_channel.to_numpy()

    # load the behavioral data
    #folder_name = "trial_info"
    #bhv_info_filename = "trial_info_2021-05-{0}.mat".format(session)
    #trial_info = scipy.io.loadmat(os.path.join(animal_folder_name,folder_name, bhv_info_filename))

    # extract information from trial_info: in matlab this is a struct with the fields:
    #trials_start_time = trial_info['trial_info'][0,0][0].ravel()
    #trials_stimulus_on = trial_info['trial_info'][0,0][1].ravel()
    #trials_response = trial_info['trial_info'][0,0][2].ravel()
    #trials_is_right_lick = trial_info['trial_info'][0,0][3].ravel()
    #trials_is_reward = trial_info['trial_info'][0,0][4].ravel()
    #trials_end_time = trial_info['trial_info'][0,0][5].ravel()
    #trials_finish_reward = trial_info['trial_info'][0,0][6].ravel()
    #n_trials = len(trials_end_time) # number of trials
    
    # load behavioural data (.pkl file)
    input_filename = "trial_info/trial_info_2021-05-{0}.pkl".format(session)
    with open(os.path.join(animal_folder_name,input_filename), 'rb') as fp:
         trial_info = pickle.load(fp)

    # extract information for trial_info: in PYTHON
    trials_start_time = np.array(trial_info['start_time'])
    trials_stimulus_on = np.array(trial_info['stimulus_on'])
    trials_response = np.array(trial_info['response'])
    trials_is_right_lick = np.array(trial_info['is_right_lick'])
    trials_is_reward = np.array(trial_info['is_reward'])
    trials_end_time = np.array(trial_info['end_trial'])
    n_trials = len(trials_end_time)

    # 500msec between start and stimulus onset
    new_start = [] # initiate empty list
    new_end = [] # initiate empty list
    new_start.append(trials_stimulus_on[0]-0.5) # append first start
    new_end.append(trials_stimulus_on[1]-0.5) # append first end
    for i in range(1,n_trials-1): # append start & end times for each trial (except last)
        new_start.append(trials_stimulus_on[i]-0.5)
        new_end.append(trials_stimulus_on[i+1]-0.5)
    new_start.append(trials_stimulus_on[n_trials-1]-0.5) # append last start
    new_end.append(trials_end_time[n_trials-1]) # append last end
    new_start = np.array(new_start) # turn list to array
    new_end = np.array(new_end)
    n_trials = len(new_end)
    
    # where was the stimulus? we don't have this info. But it can be reconstructed 
    stimulus_reconstruction = np.zeros(shape=(n_trials,))
    right_choice = np.where(trials_is_reward==1)[0]
    stimulus_reconstruction[right_choice] = trials_is_right_lick[right_choice]
    wrong_choice = np.where(trials_is_reward==0)[0]
    stimulus_reconstruction[wrong_choice] = 1 - trials_is_right_lick[wrong_choice]

    # split traces data into trials
    X = {}
    for i in range(n_trials-1):
        lower_bound = np.min(np.where(time_array>=new_start[i]))
        upper_bound = np.max(np.where(time_array<=new_end[i]))
        X[i] = channels_array[lower_bound:upper_bound,:]
        
        
    ###### across all trials for this session
    n_components = 7
    start_timepoint = 0
    n_after_start = 7
    n_before_stim = 2
    n_after_stim = 5
    n_before_resp = 5
    n_after_resp = 2
    n_before_pun = 2
    n_after_pun = 5
    n_before_rew = 2
    n_after_rew = 5
    time_window = n_before_stim+n_after_stim

    # start of trial
    variances_start = []
    for idx in X:
        x = X[idx][start_timepoint:n_after_start]
        if len(x)>6: # make sure that time window is not too small
            covar_matrix = PCA(n_components=n_components)
            covar_matrix.fit(x)
            variance = covar_matrix.explained_variance_ratio_ #calculate variance ratios
            var_trial = np.cumsum(np.round(covar_matrix.explained_variance_ratio_, decimals=3))
        variances_start.append(var_trial)
    var_start = np.mean(variances_start, axis=0)

    # stimulus onset
    variances_stim_onset = []
    for idx in X:
        stim_timepoint = int((trials_stimulus_on[idx] - new_start[idx]) /sampling_rate)
        x = X[idx][stim_timepoint-n_before_stim:stim_timepoint + n_after_stim]
        if len(x)>6: # make sure that time window is not too small
            covar_matrix = PCA(n_components=n_components)
            covar_matrix.fit(x)
            variance = covar_matrix.explained_variance_ratio_ #calculate variance ratios
            var_trial = np.cumsum(np.round(covar_matrix.explained_variance_ratio_, decimals=3))
        variances_stim_onset.append(var_trial)
    var_stim_onset = np.mean(variances_stim_onset, axis=0)

    # response
    variances_resp = []
    for idx in X:
        resp_timepoint = int((trials_response[idx] - new_start[idx]) / sampling_rate)
        x = X[idx][resp_timepoint - n_before_resp:resp_timepoint+n_after_resp]
        if len(x)>6: # make sure that time window is not too small
            covar_matrix = PCA(n_components=n_components)
            covar_matrix.fit(x)
            variance = covar_matrix.explained_variance_ratio_ #calculate variance ratios
            var_trial = np.cumsum(np.round(covar_matrix.explained_variance_ratio_, decimals=3))
        variances_resp.append(var_trial)
    var_resp = np.mean(variances_resp, axis=0)

    # punishment
    variances_punish = []
    for idx in X:
        # only punishment trials
        if trials_is_reward[idx]==0:
            # timepoint for response
            resp_timepoint = int((trials_end_time[idx] - new_start[idx]) / sampling_rate)
            x = X[idx][resp_timepoint-n_before_pun:resp_timepoint+n_after_pun]
            if len(x)>6: # make sure that time window is not too small
                covar_matrix = PCA(n_components=n_components)
                covar_matrix.fit(x)
                variance = covar_matrix.explained_variance_ratio_ #calculate variance ratios
                var_trial = np.cumsum(np.round(covar_matrix.explained_variance_ratio_, decimals=3))
        variances_punish.append(var_trial)
    var_punish = np.mean(variances_punish, axis=0)
    
    # reward
    variances_reward = []
    for idx in X:
        # only reward trials
        if trials_is_reward[idx]==1:
            # timepoint for response
            resp_timepoint = int((trials_end_time[idx] - new_start[idx]) / sampling_rate)
            x = X[idx][resp_timepoint-n_before_rew:resp_timepoint+n_after_rew]
            if len(x)>6: # make sure that time window is not too small
                covar_matrix = PCA(n_components=n_components)
                covar_matrix.fit(x)
                variance = covar_matrix.explained_variance_ratio_ #calculate variance ratios
                var_trial = np.cumsum(np.round(covar_matrix.explained_variance_ratio_, decimals=3))
        variances_reward.append(var_trial)
    var_reward = np.mean(variances_reward, axis=0)

    first_var_start.append(var_start[0])
    first_var_stim_onset.append(var_stim_onset[0])
    first_var_resp.append(var_resp[0])
    first_var_punish.append(var_punish[0])
    first_var_reward.append(var_reward[0])

In [16]:
# change variable1 and variable2 to compute permutation and t-tests between them:
# first_var_start  first_var_stim_onset  first_var_resp  first_var_punish  first_var_reward
variable1 = first_var_punish
variable2 = first_var_reward

# Compute the ground truth absolute difference Δτ between τ₁ and τ₂ from your two variables:
gT = np.abs(np.average(variable1) - np.average(variable2))

# Pool your variables into one single distribution:
pV = list(variable1) + list(variable2)

# Randomly sample (also called bootstrapping) without replacement two distributions 
# with the size equal to the original distributions from this pooled distribution 
# to compute the absolute difference RΔτ of your metric between your two permuted samples:
# and repeat this p times:

# Copy pooled distribution:
pS = copy.copy(pV)
# Initialize permutation:
pD = []
# Define p (number of permutations):
p=1000
# Permutation loop:
for i in range(0,p):
  # Shuffle the data:
    random.shuffle(pS)
    # Compute permuted absolute difference of your two sampled distributions and store it in pD:
    pD.append(np.abs(np.average(pS[0:int(len(pS)/2)]) - np.average(pS[int(len(pS)/2):])))
    
# Finally, the proportion of permuted differences higher than your ground truth difference 
# is your significance value:
p_val = len(np.where(pD>=gT)[0])/p
print("permutation test: pvalue =", p_val)
# Compare with parametric t-test:
print("parametric t-test:", scipy.stats.stats.ttest_ind(variable1, variable2))

permutation test: pvalue = 0.374
parametric t-test: Ttest_indResult(statistic=-0.8993176119907691, pvalue=0.38369122971726255)


# permutation tests: first principal component (no airpuff, trials)

In [17]:
# initiate empty lists
variances_start = []
variances_stim_onset = []
variances_resp = []
variances_punish = []
variances_reward = []
tot_trials = 0

# for all non-airpuff sessions:
for session in session_list:

    # load csv calcium traces
    folder_name = "Calcium_Traces/" 
    filename = "2021-05-{0}.csv".format(session)
    df = pd.read_csv(os.path.join(animal_folder_name,folder_name, filename))
    n_rows, n_columns = df.shape

    # create time array
    df_times = df[' '][1:-1]
    df_num_time = pd.to_numeric(df_times, downcast='float')
    time_array = df_num_time.to_numpy()
    tot_timepoints = len(time_array)

    # check time sampling - i.e. if every data point is taken at the same "distance" in time from the next - or previous
    time_sampling = np.zeros(shape=(len(time_array)-1,))
    for i in range(1,len(time_array)):
        time_sampling[i-1] = time_array[i]-time_array[i-1]

    #12420 timepoints sampled at 100msec. means that every 10 of those timepoints is 1 sec
    sampling_rate = 0.1
    tot_secs = int(tot_timepoints * sampling_rate)

    #create channels array
    n_channels = n_columns-1
    channels_array = np.zeros(shape=(tot_timepoints, n_channels)) # initiate empty array
    # fill empty channels array
    for i in range(n_channels):
        col_name = df.columns[i+1]
        df_channel = df[col_name][1:-1]
        df_num_channel = pd.to_numeric(df_channel, downcast='float')
        channels_array[:,i] = df_num_channel.to_numpy()

    # load the behavioral data
    #folder_name = "trial_info"
    #bhv_info_filename = "trial_info_2021-05-{0}.mat".format(session)
    #trial_info = scipy.io.loadmat(os.path.join(animal_folder_name,folder_name, bhv_info_filename))

    # extract information from trial_info: in matlab this is a struct with the fields:
    #trials_start_time = trial_info['trial_info'][0,0][0].ravel()
    #trials_stimulus_on = trial_info['trial_info'][0,0][1].ravel()
    #trials_response = trial_info['trial_info'][0,0][2].ravel()
    #trials_is_right_lick = trial_info['trial_info'][0,0][3].ravel()
    #trials_is_reward = trial_info['trial_info'][0,0][4].ravel()
    #trials_end_time = trial_info['trial_info'][0,0][5].ravel()
    #trials_finish_reward = trial_info['trial_info'][0,0][6].ravel()
    #n_trials = len(trials_end_time) # number of trials
    #tot_trials = tot_trials + n_trials # total number of trials across sessions
    
    # load behavioural data (.pkl file)
    input_filename = "trial_info/trial_info_2021-05-{0}.pkl".format(session)
    with open(os.path.join(animal_folder_name,input_filename), 'rb') as fp:
         trial_info = pickle.load(fp)

    # extract information for trial_info: in PYTHON
    trials_start_time = np.array(trial_info['start_time'])
    trials_stimulus_on = np.array(trial_info['stimulus_on'])
    trials_response = np.array(trial_info['response'])
    trials_is_right_lick = np.array(trial_info['is_right_lick'])
    trials_is_reward = np.array(trial_info['is_reward'])
    trials_end_time = np.array(trial_info['end_trial'])
    n_trials = len(trials_end_time)
    tot_trials = tot_trials + n_trials # total number of trials across sessions

    # 500msec between start and stimulus onset
    new_start = [] # initiate empty list
    new_end = [] # initiate empty list
    new_start.append(trials_stimulus_on[0]-0.5) # append first start
    new_end.append(trials_stimulus_on[1]-0.5) # append first end
    for i in range(1,n_trials-1): # append start & end times for each trial (except last)
        new_start.append(trials_stimulus_on[i]-0.5)
        new_end.append(trials_stimulus_on[i+1]-0.5)
    new_start.append(trials_stimulus_on[n_trials-1]-0.5) # append last start
    new_end.append(trials_end_time[n_trials-1]) # append last end
    new_start = np.array(new_start) # turn list to array
    new_end = np.array(new_end)
    n_trials = len(new_end)
    
    # where was the stimulus? we don't have this info. But it can be reconstructed 
    stimulus_reconstruction = np.zeros(shape=(n_trials,))
    right_choice = np.where(trials_is_reward==1)[0]
    stimulus_reconstruction[right_choice] = trials_is_right_lick[right_choice]
    wrong_choice = np.where(trials_is_reward==0)[0]
    stimulus_reconstruction[wrong_choice] = 1 - trials_is_right_lick[wrong_choice]

    # split traces data by trials
    X = {}
    for i in range(n_trials-1):
        lower_bound = np.min(np.where(time_array>=new_start[i]))
        upper_bound = np.max(np.where(time_array<=new_end[i]))
        X[i] = channels_array[lower_bound:upper_bound,:]
        
        
    ###### across all trials for this session
    n_components = 7
    start_timepoint = 0
    n_after_start = 7
    n_before_stim = 2
    n_after_stim = 5
    n_before_resp = 5
    n_after_resp = 2
    n_before_pun = 2
    n_after_pun = 5
    n_before_rew = 2
    n_after_rew = 5
    time_window = n_before_stim+n_after_stim

    # start of trial
    for idx in X:
        x = X[idx][start_timepoint:n_after_start]
        if len(x)>6: # make sure that time window is not too small
            covar_matrix = PCA(n_components=n_components)
            covar_matrix.fit(x)
            variance = covar_matrix.explained_variance_ratio_ #calculate variance ratios
            var_trial = np.cumsum(np.round(covar_matrix.explained_variance_ratio_, decimals=3))
        variances_start.append(var_trial[0])

    # stimulus onset
    for idx in X:
        stim_timepoint = int((trials_stimulus_on[idx] - new_start[idx]) /sampling_rate)
        x = X[idx][stim_timepoint-n_before_stim:stim_timepoint + n_after_stim]
        if len(x)>6: # make sure that time window is not too small
            covar_matrix = PCA(n_components=n_components)
            covar_matrix.fit(x)
            variance = covar_matrix.explained_variance_ratio_ #calculate variance ratios
            var_trial = np.cumsum(np.round(covar_matrix.explained_variance_ratio_, decimals=3))
        variances_stim_onset.append(var_trial[0])

    # response
    for idx in X:
        resp_timepoint = int((trials_response[idx] - new_start[idx]) / sampling_rate)
        x = X[idx][resp_timepoint - n_before_resp:resp_timepoint+n_after_resp]
        if len(x)>6: # make sure that time window is not too small
            covar_matrix = PCA(n_components=n_components)
            covar_matrix.fit(x)
            variance = covar_matrix.explained_variance_ratio_ #calculate variance ratios
            var_trial = np.cumsum(np.round(covar_matrix.explained_variance_ratio_, decimals=3))
        variances_resp.append(var_trial[0])

    # punishment
    for idx in X:
        # only punishment trials
        if trials_is_reward[idx]==0:
            # timepoint for response
            resp_timepoint = int((trials_end_time[idx] - new_start[idx]) / sampling_rate)
            x = X[idx][resp_timepoint-n_before_pun:resp_timepoint+n_after_pun]
            if len(x)>6: # make sure that time window is not too small
                covar_matrix = PCA(n_components=n_components)
                covar_matrix.fit(x)
                variance = covar_matrix.explained_variance_ratio_ #calculate variance ratios
                var_trial = np.cumsum(np.round(covar_matrix.explained_variance_ratio_, decimals=3))
        variances_punish.append(var_trial[0])
    
    # reward
    for idx in X:
        # only reward trials
        if trials_is_reward[idx]==1:
            # timepoint for response
            resp_timepoint = int((trials_end_time[idx] - new_start[idx]) / sampling_rate)
            x = X[idx][resp_timepoint-n_before_rew:resp_timepoint+n_after_rew]
            if len(x)>6: # make sure that time window is not too small
                covar_matrix = PCA(n_components=n_components)
                covar_matrix.fit(x)
                variance = covar_matrix.explained_variance_ratio_ #calculate variance ratios
                var_trial = np.cumsum(np.round(covar_matrix.explained_variance_ratio_, decimals=3))
        variances_reward.append(var_trial[0])

In [18]:
# change variable1 and variable2 to compute permutation and t-tests between them:
# variances_start  variances_stim_onset  variances_resp  variances_punish  variances_reward
variable1 = variances_punish
variable2 = variances_reward

# Compute the ground truth absolute difference Δτ between τ₁ and τ₂ from your two variables:
gT = np.abs(np.average(variable1) - np.average(variable2))

# Pool your variables into one single distribution:
pV = list(variable1) + list(variable2)

# Randomly sample (also called bootstrapping) without replacement two distributions 
# with the size equal to the original distributions from this pooled distribution 
# to compute the absolute difference RΔτ of your metric between your two permuted samples:
# and repeat this p times:

# Copy pooled distribution:
pS = copy.copy(pV)
# Initialize permutation:
pD = []
# Define p (number of permutations):
p=1000
# Permutation loop:
for i in range(0,p):
  # Shuffle the data:
    random.shuffle(pS)
    # Compute permuted absolute difference of your two sampled distributions and store it in pD:
    pD.append(np.abs(np.average(pS[0:int(len(pS)/2)]) - np.average(pS[int(len(pS)/2):])))
    
# Finally, the proportion of permuted differences higher than your ground truth difference 
# is your significance value:
p_val = len(np.where(pD>=gT)[0])/p
print("permutation test: pvalue =", p_val)
# Compare with parametric t-test:
print("parametric t-test:", scipy.stats.stats.ttest_ind(variable1, variable2))

permutation test: pvalue = 0.218
parametric t-test: Ttest_indResult(statistic=-1.201537438182991, pvalue=0.2297387329783271)


# permutation tests: airpuff vs no airpuff (sessions)

In [19]:
### airpuff ###
# initiate empty lists
first_var_start_ap = []
first_var_stim_onset_ap = []
first_var_resp_ap = []
first_var_punish_ap = []
first_var_reward_ap = []

# for all airpuff sessions:
for session in range(14,20):

    # LOAD CSV CALCIUM TRACES
    folder_name = "Calcium_Traces/" 
    filename = "2021-05-{0}.csv".format(session)
    df = pd.read_csv(os.path.join(animal_folder_name,folder_name, filename))
    n_rows, n_columns = df.shape

    # create time array
    df_times = df[' '][1:-1]
    df_num_time = pd.to_numeric(df_times, downcast='float')
    time_array = df_num_time.to_numpy()
    tot_timepoints = len(time_array)

    # check time sampling - i.e. if every data point is taken at the same "distance" in time from the next - or previous
    time_sampling = np.zeros(shape=(len(time_array)-1,))

    #12420 timepoints sampled at 100msec. means that every 10 of those timepoints is 1 sec
    sampling_rate = 0.1
    tot_secs = int(tot_timepoints * sampling_rate)

    #create channels array
    n_channels = n_columns-1
    channels_array = np.zeros(shape=(tot_timepoints, n_channels)) # initiate empty array
    # fill empty channels array
    for i in range(n_channels):
        col_name = df.columns[i+1]
        df_channel = df[col_name][1:-1]
        df_num_channel = pd.to_numeric(df_channel, downcast='float')
        channels_array[:,i] = df_num_channel.to_numpy()

    # LOAD BEHAVIOURAL DATA
    #folder_name = "trial_info"
    #bhv_info_filename = "trial_info_2021-05-{0}.mat".format(session)
    #trial_info = scipy.io.loadmat(os.path.join(animal_folder_name,folder_name, bhv_info_filename))

    # extract information from trial_info: in matlab this is a struct with the fields:
    #trials_start_time = trial_info['trial_info'][0,0][0].ravel()
    #trials_stimulus_on = trial_info['trial_info'][0,0][1].ravel()
    #trials_response = trial_info['trial_info'][0,0][2].ravel()
    #trials_is_right_lick = trial_info['trial_info'][0,0][3].ravel()
    #trials_is_reward = trial_info['trial_info'][0,0][4].ravel()
    #trials_end_time = trial_info['trial_info'][0,0][5].ravel()
    #n_trials = len(trials_end_time) # number of trials
    
    # load behavioural data (.pkl file)
    input_filename = "trial_info/trial_info_2021-05-{0}.pkl".format(session)
    with open(os.path.join(animal_folder_name,input_filename), 'rb') as fp:
         trial_info = pickle.load(fp)

    # extract information for trial_info: in PYTHON
    trials_start_time = np.array(trial_info['start_time'])
    trials_stimulus_on = np.array(trial_info['stimulus_on'])
    trials_response = np.array(trial_info['response'])
    trials_is_right_lick = np.array(trial_info['is_right_lick'])
    trials_is_reward = np.array(trial_info['is_reward'])
    trials_end_time = np.array(trial_info['end_trial'])
    n_trials = len(trials_end_time)

    # 500msec between start and stimulus onset
    new_start = [] # initiate empty list
    new_end = [] # initiate empty list
    new_start.append(trials_stimulus_on[0]-0.5) # append first start
    new_end.append(trials_stimulus_on[1]-0.5) # append first end
    for i in range(1,n_trials-1): # append start & end times for each trial (except last)
        new_start.append(trials_stimulus_on[i]-0.5)
        new_end.append(trials_stimulus_on[i+1]-0.5)
    new_start.append(trials_stimulus_on[n_trials-1]-0.5) # append last start
    new_end.append(trials_end_time[n_trials-1]) # append last end
    new_start = np.array(new_start) # turn list to array
    new_end = np.array(new_end)
    n_trials = len(new_end)
    
    # where was the stimulus? we don't have this info. But it can be reconstructed 
    stimulus_reconstruction = np.zeros(shape=(n_trials,))
    right_choice = np.where(trials_is_reward==1)[0]
    stimulus_reconstruction[right_choice] = trials_is_right_lick[right_choice]
    wrong_choice = np.where(trials_is_reward==0)[0]
    stimulus_reconstruction[wrong_choice] = 1 - trials_is_right_lick[wrong_choice]

    # split traces data by trials
    X = {}
    for i in range(n_trials-1):
        lower_bound = np.min(np.where(time_array>=new_start[i]))
        upper_bound = np.max(np.where(time_array<=new_end[i]))
        X[i] = channels_array[lower_bound:upper_bound,:]
        
        
    ###### across all trials for this session
    n_components = 7
    start_timepoint = 0
    n_after_start = 7
    n_before_stim = 2
    n_after_stim = 5
    n_before_resp = 5
    n_after_resp = 2
    n_before_pun = 2
    n_after_pun = 5
    n_before_rew = 2
    n_after_rew = 5
    time_window = n_before_stim+n_after_stim
    airpuff = 2

    # start of trial
    variances_start = []
    for idx in X:
        x = X[idx][start_timepoint:n_after_start]
        if len(x)>6: # make sure that time window is not too small
            covar_matrix = PCA(n_components=n_components)
            covar_matrix.fit(x)
            variance = covar_matrix.explained_variance_ratio_ #calculate variance ratios
            var_trial = np.cumsum(np.round(covar_matrix.explained_variance_ratio_, decimals=3))
        variances_start.append(var_trial)
    var_start = np.mean(variances_start, axis=0)

    # stimulus onset
    variances_stim_onset = []
    for idx in X:
        stim_timepoint = int((trials_stimulus_on[idx] - new_start[idx]) /sampling_rate)
        x = X[idx][stim_timepoint-n_before_stim:stim_timepoint + n_after_stim]
        if len(x)>6: # make sure that time window is not too small
            covar_matrix = PCA(n_components=n_components)
            covar_matrix.fit(x)
            variance = covar_matrix.explained_variance_ratio_ #calculate variance ratios
            var_trial = np.cumsum(np.round(covar_matrix.explained_variance_ratio_, decimals=3))
        variances_stim_onset.append(var_trial)
    var_stim_onset = np.mean(variances_stim_onset, axis=0)

    # response
    variances_resp = []
    for idx in X:
        resp_timepoint = int((trials_response[idx] - new_start[idx]) / sampling_rate)
        x = X[idx][resp_timepoint - n_before_resp:resp_timepoint+n_after_resp]
        if len(x)>6: # make sure that time window is not too small
            covar_matrix = PCA(n_components=n_components)
            covar_matrix.fit(x)
            variance = covar_matrix.explained_variance_ratio_ #calculate variance ratios
            var_trial = np.cumsum(np.round(covar_matrix.explained_variance_ratio_, decimals=3))
        variances_resp.append(var_trial)
    var_resp = np.mean(variances_resp, axis=0)

    # punishment
    variances_punish = []
    for idx in X:
        # only punishment trials
        if trials_is_reward[idx]==0:
            # timepoint for response
            resp_airpuff_timepoint = int((trials_response[idx] - new_start[idx]) / sampling_rate) + airpuff
            x = X[idx][resp_timepoint-n_before_pun:resp_timepoint+n_after_pun]
            if len(x)>6: # make sure that time window is not too small
                covar_matrix = PCA(n_components=n_components)
                covar_matrix.fit(x)
                variance = covar_matrix.explained_variance_ratio_ #calculate variance ratios
                var_trial = np.cumsum(np.round(covar_matrix.explained_variance_ratio_, decimals=3))
        variances_punish.append(var_trial)
    var_punish = np.mean(variances_punish, axis=0)
    
    # reward
    variances_reward = []
    for idx in X:
        # only reward trials
        if trials_is_reward[idx]==1:
            # timepoint for response
            resp_timepoint = int((trials_end_time[idx] - new_start[idx]) / sampling_rate)
            x = X[idx][resp_timepoint-n_before_rew:resp_timepoint+n_after_rew]
            if len(x)>6: # make sure that time window is not too small
                covar_matrix = PCA(n_components=n_components)
                covar_matrix.fit(x)
                variance = covar_matrix.explained_variance_ratio_ #calculate variance ratios
                var_trial = np.cumsum(np.round(covar_matrix.explained_variance_ratio_, decimals=3))
        variances_reward.append(var_trial)
    var_reward = np.mean(variances_reward, axis=0)

    first_var_start_ap.append(var_start[0])
    first_var_stim_onset_ap.append(var_stim_onset[0])
    first_var_resp_ap.append(var_resp[0])
    first_var_punish_ap.append(var_punish[0])
    first_var_reward_ap.append(var_reward[0])

In [20]:
### no airpuff ###
# initiate empty lists
first_var_start = []
first_var_stim_onset = []
first_var_resp = []
first_var_punish = []
first_var_reward = []

# for all non-airpuff sessions:
for session in session_list:

    # load csv calcium traces
    folder_name = "Calcium_Traces/" 
    filename = "2021-05-{0}.csv".format(session)
    df = pd.read_csv(os.path.join(animal_folder_name,folder_name, filename))
    n_rows, n_columns = df.shape

    # create time array
    df_times = df[' '][1:-1]
    df_num_time = pd.to_numeric(df_times, downcast='float')
    time_array = df_num_time.to_numpy()
    tot_timepoints = len(time_array)

    # check time sampling - i.e. if every data point is taken at the same "distance" in time from the next - or previous
    time_sampling = np.zeros(shape=(len(time_array)-1,))
    for i in range(1,len(time_array)):
        time_sampling[i-1] = time_array[i]-time_array[i-1]

    #12420 timepoints sampled at 100msec. means that every 10 of those timepoints is 1 sec
    sampling_rate = 0.1
    tot_secs = int(tot_timepoints * sampling_rate)

    #create channels array
    n_channels = n_columns-1
    channels_array = np.zeros(shape=(tot_timepoints, n_channels)) # initiate empty array
    # fill empty channels array
    for i in range(n_channels):
        col_name = df.columns[i+1]
        df_channel = df[col_name][1:-1]
        df_num_channel = pd.to_numeric(df_channel, downcast='float')
        channels_array[:,i] = df_num_channel.to_numpy()

    # load the behavioral data
    #folder_name = "trial_info"
    #bhv_info_filename = "trial_info_2021-05-{0}.mat".format(session)
    #trial_info = scipy.io.loadmat(os.path.join(animal_folder_name,folder_name, bhv_info_filename))

    # extract information from trial_info: in matlab this is a struct with the fields:
    #trials_start_time = trial_info['trial_info'][0,0][0].ravel()
    #trials_stimulus_on = trial_info['trial_info'][0,0][1].ravel()
    #trials_response = trial_info['trial_info'][0,0][2].ravel()
    #trials_is_right_lick = trial_info['trial_info'][0,0][3].ravel()
    #trials_is_reward = trial_info['trial_info'][0,0][4].ravel()
    #trials_end_time = trial_info['trial_info'][0,0][5].ravel()
    #trials_finish_reward = trial_info['trial_info'][0,0][6].ravel()
    #n_trials = len(trials_end_time) # number of trials
    
    # load behavioural data (.pkl file)
    input_filename = "trial_info/trial_info_2021-05-{0}.pkl".format(session)
    with open(os.path.join(animal_folder_name,input_filename), 'rb') as fp:
         trial_info = pickle.load(fp)

    # extract information for trial_info: in PYTHON
    trials_start_time = np.array(trial_info['start_time'])
    trials_stimulus_on = np.array(trial_info['stimulus_on'])
    trials_response = np.array(trial_info['response'])
    trials_is_right_lick = np.array(trial_info['is_right_lick'])
    trials_is_reward = np.array(trial_info['is_reward'])
    trials_end_time = np.array(trial_info['end_trial'])
    n_trials = len(trials_end_time)

    # 500msec between start and stimulus onset
    new_start = [] # initiate empty list
    new_end = [] # initiate empty list
    new_start.append(trials_stimulus_on[0]-0.5) # append first start
    new_end.append(trials_stimulus_on[1]-0.5) # append first end
    for i in range(1,n_trials-1): # append start & end times for each trial (except last)
        new_start.append(trials_stimulus_on[i]-0.5)
        new_end.append(trials_stimulus_on[i+1]-0.5)
    new_start.append(trials_stimulus_on[n_trials-1]-0.5) # append last start
    new_end.append(trials_end_time[n_trials-1]) # append last end
    new_start = np.array(new_start) # turn list to array
    new_end = np.array(new_end)
    n_trials = len(new_end)
    
    # where was the stimulus? we don't have this info. But it can be reconstructed 
    stimulus_reconstruction = np.zeros(shape=(n_trials,))
    right_choice = np.where(trials_is_reward==1)[0]
    stimulus_reconstruction[right_choice] = trials_is_right_lick[right_choice]
    wrong_choice = np.where(trials_is_reward==0)[0]
    stimulus_reconstruction[wrong_choice] = 1 - trials_is_right_lick[wrong_choice]

    # split traces data by trials
    X = {}
    for i in range(n_trials-1):
        lower_bound = np.min(np.where(time_array>=new_start[i]))
        upper_bound = np.max(np.where(time_array<=new_end[i]))
        X[i] = channels_array[lower_bound:upper_bound,:]
        
        
    ###### across all trials for this session
    n_components = 7
    start_timepoint = 0
    n_after_start = 7
    n_before_stim = 2
    n_after_stim = 5
    n_before_resp = 5
    n_after_resp = 2
    n_before_pun = 2
    n_after_pun = 5
    n_before_rew = 2
    n_after_rew = 5
    time_window = n_before_stim+n_after_stim

    # start of trial
    variances_start = []
    for idx in X:
        x = X[idx][start_timepoint:n_after_start]
        if len(x)>6: # make sure that time window is not too small
            covar_matrix = PCA(n_components=n_components)
            covar_matrix.fit(x)
            variance = covar_matrix.explained_variance_ratio_ #calculate variance ratios
            var_trial = np.cumsum(np.round(covar_matrix.explained_variance_ratio_, decimals=3))
        variances_start.append(var_trial)
    var_start = np.mean(variances_start, axis=0)

    # stimulus onset
    variances_stim_onset = []
    for idx in X:
        stim_timepoint = int((trials_stimulus_on[idx] - new_start[idx]) /sampling_rate)
        x = X[idx][stim_timepoint-n_before_stim:stim_timepoint + n_after_stim]
        if len(x)>6: # make sure that time window is not too small
            covar_matrix = PCA(n_components=n_components)
            covar_matrix.fit(x)
            variance = covar_matrix.explained_variance_ratio_ #calculate variance ratios
            var_trial = np.cumsum(np.round(covar_matrix.explained_variance_ratio_, decimals=3))
        variances_stim_onset.append(var_trial)
    var_stim_onset = np.mean(variances_stim_onset, axis=0)

    # response
    variances_resp = []
    for idx in X:
        resp_timepoint = int((trials_response[idx] - new_start[idx]) / sampling_rate)
        x = X[idx][resp_timepoint - n_before_resp:resp_timepoint+n_after_resp]
        if len(x)>6: # make sure that time window is not too small
            covar_matrix = PCA(n_components=n_components)
            covar_matrix.fit(x)
            variance = covar_matrix.explained_variance_ratio_ #calculate variance ratios
            var_trial = np.cumsum(np.round(covar_matrix.explained_variance_ratio_, decimals=3))
        variances_resp.append(var_trial)
    var_resp = np.mean(variances_resp, axis=0)

    # punishment
    variances_punish = []
    for idx in X:
        # only punishment trials
        if trials_is_reward[idx]==0:
            # timepoint for response
            resp_timepoint = int((trials_end_time[idx] - new_start[idx]) / sampling_rate)
            x = X[idx][resp_timepoint-n_before_pun:resp_timepoint+n_after_pun]
            if len(x)>6: # make sure that time window is not too small
                covar_matrix = PCA(n_components=n_components)
                covar_matrix.fit(x)
                variance = covar_matrix.explained_variance_ratio_ #calculate variance ratios
                var_trial = np.cumsum(np.round(covar_matrix.explained_variance_ratio_, decimals=3))
        variances_punish.append(var_trial)
    var_punish = np.mean(variances_punish, axis=0)
    
    # reward
    variances_reward = []
    for idx in X:
        # only reward trials
        if trials_is_reward[idx]==1:
            # timepoint for response
            resp_timepoint = int((trials_end_time[idx] - new_start[idx]) / sampling_rate)
            x = X[idx][resp_timepoint-n_before_rew:resp_timepoint+n_after_rew]
            if len(x)>6: # make sure that time window is not too small
                covar_matrix = PCA(n_components=n_components)
                covar_matrix.fit(x)
                variance = covar_matrix.explained_variance_ratio_ #calculate variance ratios
                var_trial = np.cumsum(np.round(covar_matrix.explained_variance_ratio_, decimals=3))
        variances_reward.append(var_trial)
    var_reward = np.mean(variances_reward, axis=0)

    first_var_start.append(var_start[0])
    first_var_stim_onset.append(var_stim_onset[0])
    first_var_resp.append(var_resp[0])
    first_var_punish.append(var_punish[0])
    first_var_reward.append(var_reward[0])

In [21]:
# change variable1 and variable2 to compute permutation and t-tests between them:
# airpuff: first_var_start_ap  first_var_stim_onset_ap  first_var_resp_ap  first_var_punish_ap  first_var_reward_ap
# no airpuff: first_var_start  first_var_stim_onset  first_var_resp  first_var_punish  first_var_reward
variable1 = first_var_reward_ap
variable2 = first_var_reward

# Compute the ground truth absolute difference Δτ between τ₁ and τ₂ from your two variables:
gT = np.abs(np.average(variable1) - np.average(variable2))

# Pool your variables into one single distribution:
pV = list(variable1) + list(variable2)

# Randomly sample (also called bootstrapping) without replacement two distributions 
# with the size equal to the original distributions from this pooled distribution 
# to compute the absolute difference RΔτ of your metric between your two permuted samples:
# and repeat this p times:

# Copy pooled distribution:
pS = copy.copy(pV)
# Initialize permutation:
pD = []
# Define p (number of permutations):
p=1000
# Permutation loop:
for i in range(0,p):
  # Shuffle the data:
    random.shuffle(pS)
    # Compute permuted absolute difference of your two sampled distributions and store it in pD:
    pD.append(np.abs(np.average(pS[0:int(len(pS)/2)]) - np.average(pS[int(len(pS)/2):])))
    
# Finally, the proportion of permuted differences higher than your ground truth difference 
# is your significance value:
p_val = len(np.where(pD>=gT)[0])/p
print("permutation test: pvalue =", p_val)
# Compare with parametric t-test:
print("parametric t-test:", scipy.stats.stats.ttest_ind(variable1, variable2))

permutation test: pvalue = 0.46
parametric t-test: Ttest_indResult(statistic=-0.7800516640808547, pvalue=0.45047298680038195)


# permutation tests: airpuff vs no airpuff (trials)

In [22]:
### no airpuff ###
# initiate empty lists
variances_start = []
variances_stim_onset = []
variances_resp = []
variances_punish = []
variances_reward = []
tot_trials = 0

# for all non-airpuff sessions:
for session in session_list:

    # load csv calcium traces
    folder_name = "Calcium_Traces/" 
    filename = "2021-05-{0}.csv".format(session)
    df = pd.read_csv(os.path.join(animal_folder_name,folder_name, filename))
    n_rows, n_columns = df.shape

    # create time array
    df_times = df[' '][1:-1]
    df_num_time = pd.to_numeric(df_times, downcast='float')
    time_array = df_num_time.to_numpy()
    tot_timepoints = len(time_array)

    # check time sampling - i.e. if every data point is taken at the same "distance" in time from the next - or previous
    time_sampling = np.zeros(shape=(len(time_array)-1,))
    for i in range(1,len(time_array)):
        time_sampling[i-1] = time_array[i]-time_array[i-1]

    #12420 timepoints sampled at 100msec. means that every 10 of those timepoints is 1 sec
    sampling_rate = 0.1
    tot_secs = int(tot_timepoints * sampling_rate)

    #create channels array
    n_channels = n_columns-1
    channels_array = np.zeros(shape=(tot_timepoints, n_channels)) # initiate empty array
    # fill empty channels array
    for i in range(n_channels):
        col_name = df.columns[i+1]
        df_channel = df[col_name][1:-1]
        df_num_channel = pd.to_numeric(df_channel, downcast='float')
        channels_array[:,i] = df_num_channel.to_numpy()

    # load the behavioral data
    #folder_name = "trial_info"
    #bhv_info_filename = "trial_info_2021-05-{0}.mat".format(session)
    #trial_info = scipy.io.loadmat(os.path.join(animal_folder_name,folder_name, bhv_info_filename))

    # extract information from trial_info: in matlab this is a struct with the fields:
    #trials_start_time = trial_info['trial_info'][0,0][0].ravel()
    #trials_stimulus_on = trial_info['trial_info'][0,0][1].ravel()
    #trials_response = trial_info['trial_info'][0,0][2].ravel()
    #trials_is_right_lick = trial_info['trial_info'][0,0][3].ravel()
    #trials_is_reward = trial_info['trial_info'][0,0][4].ravel()
    #trials_end_time = trial_info['trial_info'][0,0][5].ravel()
    #trials_finish_reward = trial_info['trial_info'][0,0][6].ravel()
    #n_trials = len(trials_end_time) # number of trials
    #tot_trials = tot_trials + n_trials # total number of trials across sessions
    
    # load behavioural data (.pkl file)
    input_filename = "trial_info/trial_info_2021-05-{0}.pkl".format(session)
    with open(os.path.join(animal_folder_name,input_filename), 'rb') as fp:
         trial_info = pickle.load(fp)

    # extract information for trial_info: in PYTHON
    trials_start_time = np.array(trial_info['start_time'])
    trials_stimulus_on = np.array(trial_info['stimulus_on'])
    trials_response = np.array(trial_info['response'])
    trials_is_right_lick = np.array(trial_info['is_right_lick'])
    trials_is_reward = np.array(trial_info['is_reward'])
    trials_end_time = np.array(trial_info['end_trial'])
    n_trials = len(trials_end_time)
    tot_trials = tot_trials + n_trials # total number of trials across sessions

    # 500msec between start and stimulus onset
    new_start = [] # initiate empty list
    new_end = [] # initiate empty list
    new_start.append(trials_stimulus_on[0]-0.5) # append first start
    new_end.append(trials_stimulus_on[1]-0.5) # append first end
    for i in range(1,n_trials-1): # append start & end times for each trial (except last)
        new_start.append(trials_stimulus_on[i]-0.5)
        new_end.append(trials_stimulus_on[i+1]-0.5)
    new_start.append(trials_stimulus_on[n_trials-1]-0.5) # append last start
    new_end.append(trials_end_time[n_trials-1]) # append last end
    new_start = np.array(new_start) # turn list to array
    new_end = np.array(new_end)
    n_trials = len(new_end)
    
    # where was the stimulus? we don't have this info. But it can be reconstructed 
    stimulus_reconstruction = np.zeros(shape=(n_trials,))
    right_choice = np.where(trials_is_reward==1)[0]
    stimulus_reconstruction[right_choice] = trials_is_right_lick[right_choice]
    wrong_choice = np.where(trials_is_reward==0)[0]
    stimulus_reconstruction[wrong_choice] = 1 - trials_is_right_lick[wrong_choice]

    # split traces data by trials
    X = {}
    for i in range(n_trials-1):
        lower_bound = np.min(np.where(time_array>=new_start[i]))
        upper_bound = np.max(np.where(time_array<=new_end[i]))
        X[i] = channels_array[lower_bound:upper_bound,:]
        
        
    ###### across all trials for this session
    n_components = 7
    start_timepoint = 0
    n_after_start = 7
    n_before_stim = 2
    n_after_stim = 5
    n_before_resp = 5
    n_after_resp = 2
    n_before_pun = 2
    n_after_pun = 5
    n_before_rew = 2
    n_after_rew = 5
    time_window = n_before_stim+n_after_stim

    # start of trial
    for idx in X:
        x = X[idx][start_timepoint:n_after_start]
        if len(x)>6: # make sure that time window is not too small
            covar_matrix = PCA(n_components=n_components)
            covar_matrix.fit(x)
            variance = covar_matrix.explained_variance_ratio_ #calculate variance ratios
            var_trial = np.cumsum(np.round(covar_matrix.explained_variance_ratio_, decimals=3))
        variances_start.append(var_trial[0])

    # stimulus onset
    for idx in X:
        stim_timepoint = int((trials_stimulus_on[idx] - new_start[idx]) /sampling_rate)
        x = X[idx][stim_timepoint-n_before_stim:stim_timepoint + n_after_stim]
        if len(x)>6: # make sure that time window is not too small
            covar_matrix = PCA(n_components=n_components)
            covar_matrix.fit(x)
            variance = covar_matrix.explained_variance_ratio_ #calculate variance ratios
            var_trial = np.cumsum(np.round(covar_matrix.explained_variance_ratio_, decimals=3))
        variances_stim_onset.append(var_trial[0])

    # response
    for idx in X:
        resp_timepoint = int((trials_response[idx] - new_start[idx]) / sampling_rate)
        x = X[idx][resp_timepoint - n_before_resp:resp_timepoint+n_after_resp]
        if len(x)>6: # make sure that time window is not too small
            covar_matrix = PCA(n_components=n_components)
            covar_matrix.fit(x)
            variance = covar_matrix.explained_variance_ratio_ #calculate variance ratios
            var_trial = np.cumsum(np.round(covar_matrix.explained_variance_ratio_, decimals=3))
        variances_resp.append(var_trial[0])

    # punishment
    for idx in X:
        # only punishment trials
        if trials_is_reward[idx]==0:
            # timepoint for response
            resp_timepoint = int((trials_end_time[idx] - new_start[idx]) / sampling_rate)
            x = X[idx][resp_timepoint-n_before_pun:resp_timepoint+n_after_pun]
            if len(x)>6: # make sure that time window is not too small
                covar_matrix = PCA(n_components=n_components)
                covar_matrix.fit(x)
                variance = covar_matrix.explained_variance_ratio_ #calculate variance ratios
                var_trial = np.cumsum(np.round(covar_matrix.explained_variance_ratio_, decimals=3))
        variances_punish.append(var_trial[0])
    
    # reward
    for idx in X:
        # only reward trials
        if trials_is_reward[idx]==1:
            # timepoint for response
            resp_timepoint = int((trials_end_time[idx] - new_start[idx]) / sampling_rate)
            x = X[idx][resp_timepoint-n_before_rew:resp_timepoint+n_after_rew]
            if len(x)>6: # make sure that time window is not too small
                covar_matrix = PCA(n_components=n_components)
                covar_matrix.fit(x)
                variance = covar_matrix.explained_variance_ratio_ #calculate variance ratios
                var_trial = np.cumsum(np.round(covar_matrix.explained_variance_ratio_, decimals=3))
        variances_reward.append(var_trial[0])

In [23]:
### airpuff ###
# initiate empty lists
variances_ap_start = []
variances_ap_stim_onset = []
variances_ap_resp = []
variances_ap_punish = []
variances_ap_reward = []
tot_trials = 0

# for all airpuff sessions:
for session in range(14,20):

    # LOAD CSV CALCIUM TRACES
    folder_name = "Calcium_Traces/" 
    filename = "2021-05-{0}.csv".format(session)
    df = pd.read_csv(os.path.join(animal_folder_name,folder_name, filename))
    n_rows, n_columns = df.shape

    # create time array
    df_times = df[' '][1:-1]
    df_num_time = pd.to_numeric(df_times, downcast='float')
    time_array = df_num_time.to_numpy()
    tot_timepoints = len(time_array)

    # check time sampling - i.e. if every data point is taken at the same "distance" in time from the next - or previous
    time_sampling = np.zeros(shape=(len(time_array)-1,))

    #12420 timepoints sampled at 100msec. means that every 10 of those timepoints is 1 sec
    sampling_rate = 0.1
    tot_secs = int(tot_timepoints * sampling_rate)

    #create channels array
    n_channels = n_columns-1
    channels_array = np.zeros(shape=(tot_timepoints, n_channels)) # initiate empty array
    # fill empty channels array
    for i in range(n_channels):
        col_name = df.columns[i+1]
        df_channel = df[col_name][1:-1]
        df_num_channel = pd.to_numeric(df_channel, downcast='float')
        channels_array[:,i] = df_num_channel.to_numpy()

    # LOAD BEHAVIOURAL DATA
    #folder_name = "trial_info"
    #bhv_info_filename = "trial_info_2021-05-{0}.mat".format(session)
    #trial_info = scipy.io.loadmat(os.path.join(animal_folder_name,folder_name, bhv_info_filename))

    # extract information from trial_info: in matlab this is a struct with the fields:
    #trials_start_time = trial_info['trial_info'][0,0][0].ravel()
    #trials_stimulus_on = trial_info['trial_info'][0,0][1].ravel()
    #trials_response = trial_info['trial_info'][0,0][2].ravel()
    #trials_is_right_lick = trial_info['trial_info'][0,0][3].ravel()
    #trials_is_reward = trial_info['trial_info'][0,0][4].ravel()
    #trials_end_time = trial_info['trial_info'][0,0][5].ravel()
    #n_trials = len(trials_end_time) # number of trials
    #tot_trials = tot_trials + n_trials # total number of trials across sessions
    
    # load behavioural data (.pkl file)
    input_filename = "trial_info/trial_info_2021-05-{0}.pkl".format(session)
    with open(os.path.join(animal_folder_name,input_filename), 'rb') as fp:
         trial_info = pickle.load(fp)

    # extract information for trial_info: in PYTHON
    trials_start_time = np.array(trial_info['start_time'])
    trials_stimulus_on = np.array(trial_info['stimulus_on'])
    trials_response = np.array(trial_info['response'])
    trials_is_right_lick = np.array(trial_info['is_right_lick'])
    trials_is_reward = np.array(trial_info['is_reward'])
    trials_end_time = np.array(trial_info['end_trial'])
    n_trials = len(trials_end_time)
    tot_trials = tot_trials + n_trials # total number of trials across sessions

    # 500msec between start and stimulus onset
    new_start = [] # initiate empty list
    new_end = [] # initiate empty list
    new_start.append(trials_stimulus_on[0]-0.5) # append first start
    new_end.append(trials_stimulus_on[1]-0.5) # append first end
    for i in range(1,n_trials-1): # append start & end times for each trial (except last)
        new_start.append(trials_stimulus_on[i]-0.5)
        new_end.append(trials_stimulus_on[i+1]-0.5)
    new_start.append(trials_stimulus_on[n_trials-1]-0.5) # append last start
    new_end.append(trials_end_time[n_trials-1]) # append last end
    new_start = np.array(new_start) # turn list to array
    new_end = np.array(new_end)
    n_trials = len(new_end)
    
    # where was the stimulus? we don't have this info. But it can be reconstructed 
    stimulus_reconstruction = np.zeros(shape=(n_trials,))
    right_choice = np.where(trials_is_reward==1)[0]
    stimulus_reconstruction[right_choice] = trials_is_right_lick[right_choice]
    wrong_choice = np.where(trials_is_reward==0)[0]
    stimulus_reconstruction[wrong_choice] = 1 - trials_is_right_lick[wrong_choice]

    # split traces data by trials
    X = {}
    for i in range(n_trials-1):
        lower_bound = np.min(np.where(time_array>=new_start[i]))
        upper_bound = np.max(np.where(time_array<=new_end[i]))
        X[i] = channels_array[lower_bound:upper_bound,:]
        
        
    ###### across all trials for this session
    n_components = 7
    start_timepoint = 0
    n_after_start = 7
    n_before_stim = 2
    n_after_stim = 5
    n_before_resp = 5
    n_after_resp = 2
    n_before_pun = 2
    n_after_pun = 5
    n_before_rew = 2
    n_after_rew = 5
    time_window = n_before_stim+n_after_stim
    airpuff = 2

    # start of trial
    for idx in X:
        x = X[idx][start_timepoint:n_after_start]
        if len(x)>6: # make sure that time window is not too small
            covar_matrix = PCA(n_components=n_components)
            covar_matrix.fit(x)
            variance = covar_matrix.explained_variance_ratio_ #calculate variance ratios
            var_trial = np.cumsum(np.round(covar_matrix.explained_variance_ratio_, decimals=3))
        variances_ap_start.append(var_trial[0])

    # stimulus onset
    for idx in X:
        stim_timepoint = int((trials_stimulus_on[idx] - new_start[idx]) /sampling_rate)
        x = X[idx][stim_timepoint-n_before_stim:stim_timepoint + n_after_stim]
        if len(x)>6: # make sure that time window is not too small
            covar_matrix = PCA(n_components=n_components)
            covar_matrix.fit(x)
            variance = covar_matrix.explained_variance_ratio_ #calculate variance ratios
            var_trial = np.cumsum(np.round(covar_matrix.explained_variance_ratio_, decimals=3))
        variances_ap_stim_onset.append(var_trial[0])

    # response
    for idx in X:
        resp_timepoint = int((trials_response[idx] - new_start[idx]) / sampling_rate)
        x = X[idx][resp_timepoint - n_before_resp:resp_timepoint+n_after_resp]
        if len(x)>6: # make sure that time window is not too small
            covar_matrix = PCA(n_components=n_components)
            covar_matrix.fit(x)
            variance = covar_matrix.explained_variance_ratio_ #calculate variance ratios
            var_trial = np.cumsum(np.round(covar_matrix.explained_variance_ratio_, decimals=3))
        variances_ap_resp.append(var_trial[0])

    # punishment
    for idx in X:
        # only punishment trials
        if trials_is_reward[idx]==0:
            # timepoint for response
            resp_airpuff_timepoint = int((trials_response[idx] - new_start[idx]) / sampling_rate) + airpuff
            x = X[idx][resp_timepoint-n_before_pun:resp_timepoint+n_after_pun]
            if len(x)>6: # make sure that time window is not too small
                covar_matrix = PCA(n_components=n_components)
                covar_matrix.fit(x)
                variance = covar_matrix.explained_variance_ratio_ #calculate variance ratios
                var_trial = np.cumsum(np.round(covar_matrix.explained_variance_ratio_, decimals=3))
        variances_ap_punish.append(var_trial[0])
    
    # reward
    for idx in X:
        # only reward trials
        if trials_is_reward[idx]==1:
            # timepoint for response
            resp_timepoint = int((trials_end_time[idx] - new_start[idx]) / sampling_rate)
            x = X[idx][resp_timepoint-n_before_rew:resp_timepoint+n_after_rew]
            if len(x)>6: # make sure that time window is not too small
                covar_matrix = PCA(n_components=n_components)
                covar_matrix.fit(x)
                variance = covar_matrix.explained_variance_ratio_ #calculate variance ratios
                var_trial = np.cumsum(np.round(covar_matrix.explained_variance_ratio_, decimals=3))
        variances_ap_reward.append(var_trial[0])

In [24]:
# change variable1 and variable2 to compute permutation and t-tests between them:
# airpuff: variances_ap_start  variances_ap_stim_onset  variances_ap_resp  variances_ap_punish  variances_ap_reward
# no airpuff: variances_start  variances_stim_onset  variances_resp  variances_punish  variances_reward
variable1 = variances_ap_reward
variable2 = variances_reward

# Compute the ground truth absolute difference Δτ between τ₁ and τ₂ from your two variables:
gT = np.abs(np.average(variable1) - np.average(variable2))

# Pool your variables into one single distribution:
pV = list(variable1) + list(variable2)

# Randomly sample (also called bootstrapping) without replacement two distributions 
# with the size equal to the original distributions from this pooled distribution 
# to compute the absolute difference RΔτ of your metric between your two permuted samples:
# and repeat this p times:

# Copy pooled distribution:
pS = copy.copy(pV)
# Initialize permutation:
pD = []
# Define p (number of permutations):
p=1000
# Permutation loop:
for i in range(0,p):
  # Shuffle the data:
    random.shuffle(pS)
    # Compute permuted absolute difference of your two sampled distributions and store it in pD:
    pD.append(np.abs(np.average(pS[0:int(len(pS)/2)]) - np.average(pS[int(len(pS)/2):])))
    
# Finally, the proportion of permuted differences higher than your ground truth difference 
# is your significance value:
p_val = len(np.where(pD>=gT)[0])/p
print("permutation test: pvalue =", p_val)
# Compare with parametric t-test:
print("parametric t-test:", scipy.stats.stats.ttest_ind(variable1, variable2))

permutation test: pvalue = 0.037
parametric t-test: Ttest_indResult(statistic=-1.9120858534666179, pvalue=0.0560910493073145)
